# This work employs deep convolution neural network model to distinguish between fresh and rotten products in three popular fruits, apple, orange and banana. An accuracy of 99% was reached with both VGG16 and ResNet101 models

In [8]:
# Importing necessary modules
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import random
import os

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from PIL import Image 


from keras.applications.vgg16 import VGG16, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

from sklearn.metrics import confusion_matrix,classification_report

In [9]:
# Image processing libraries
!pip install imutils
import cv2
import os
from imutils import paths

Defaulting to user installation because normal site-packages is not writeable


In [10]:
#Fetching the fresh train and validation data.
import os
FRESH_PATH = "C:\\Users\\USER\\Desktop\\algo"
FRESH_TRAIN_PATH = os.path.sep.join([FRESH_PATH, "train"])
FRESH_TEST_PATH = os.path.sep.join([FRESH_PATH, "test"])

# Get total number of images per class
freshTotalTrain = len(list(paths.list_images(FRESH_TRAIN_PATH)))
freshTotalTest = len(list(paths.list_images(FRESH_TEST_PATH)))

In [11]:
#Creating the CLAHE Function
clahe = cv2.createCLAHE(clipLimit=10,tileGridSize=(8,8))

def apply_clahe(img):
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    img[:,:,0] = clahe.apply(img[:,:,0])
    img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB)
    return img

## Apply CLAHE to all images and save to the destination directory

In [13]:
import os
import cv2

# Define the source directory containing the images
source_dir = "C:\\Users\\USER\\Desktop\\algo\\dataset"

# Define the destination directory to save the processed images
destination_dir = "C:\\Users\\USER\\Desktop\\algo\\dataset2"

# Define a function to apply CLAHE on the image
def apply_clahe(image):
    return processed_image

for subdir, dirs, files in os.walk(source_dir):
    for file in files:
        # Get the full file path
        file_path = os.path.join(subdir, file)
        
        # Load the image using OpenCV
        image = cv2.imread(file_path)
        
        if image is not None:
            # Apply the custom function to process the image
            processed_image = apply_clahe(image)
            
            # Get the relative path of the image with respect to the source directory
            relative_path = os.path.relpath(file_path, source_dir)
            
            # Construct the destination path to save the processed image
            destination_path = os.path.join(destination_dir, relative_path)
            
            # Create the destination directory if it does not exist
            os.makedirs(os.path.dirname(destination_path), exist_ok=True)
            
            # Save the processed image to the destination directory
            cv2.imwrite(destination_path, processed_image)
        else:
            print(f"Error loading image: {file_path}")


Error loading image: C:\Users\USER\Desktop\algo\dataset\train\rotteng\다운로드 (1).jpg
Error loading image: C:\Users\USER\Desktop\algo\dataset\train\rotteng\다운로드 (2).jpg
Error loading image: C:\Users\USER\Desktop\algo\dataset\train\rotteng\다운로드 (3).jpg
Error loading image: C:\Users\USER\Desktop\algo\dataset\train\rotteng\다운로드 (4).jpg
Error loading image: C:\Users\USER\Desktop\algo\dataset\train\rotteng\다운로드.jpg


In [14]:
#Fetch CLAHE-converted images
import os
BASE_PATH = "C:\\Users\\USER\\Desktop\\algo\\dataset2"
TRAIN_PATH = os.path.sep.join([BASE_PATH, "train"])
TEST_PATH = os.path.sep.join([BASE_PATH, "test"])

# Get total number of images per class
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 증강을 위한 데이터 생성기 설정
trainAug = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# 훈련용 및 검증용 데이터셋 생성
train_generator = trainAug.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=32,
    subset="training"
)

val_generator = trainAug.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=32,
    subset="validation"
)


Found 1376 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


## Show comparison between fresh and CLAHE-applied images

In [20]:
import shutil
shutil.make_archive('clahe-dataset', 'zip', destination_dir)

'C:\\Users\\USER\\Desktop\\clahe-dataset.zip'

In [21]:
# Getting labels of training data
labels = os.listdir(TRAIN_PATH)
print("Classes names:")
print(".............")
labels

Classes names:
.............


['freshg', 'rotteng']

In [22]:
# Initialize the training data for augmentation 
trainAug= ImageDataGenerator(
    validation_split = 0.2,
	rescale=1./255,
	rotation_range=25,
	zoom_range=0.2,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")

testAug= ImageDataGenerator(rescale=1. / 255)

In [23]:
# initialize the training generator
train_generator = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=32,
    subset = "training")

val_generator = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=32,
    subset = "validation")


# initialize the validation generator
test_generator = testAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False)

Found 1376 images belonging to 2 classes.
Found 344 images belonging to 2 classes.
Found 332 images belonging to 2 classes.


## Creating the utility model evaluation function

In [25]:
import seaborn as sns
from matplotlib import pyplot

def data_viz(model, history):
  pyplot.style.use("ggplot")
  # sns.set()
  fig = pyplot.figure(0, (12, 4))

  ax = pyplot.subplot(1, 2, 1)
  pyplot.plot(np.arange(0, len(history.epoch)), history.history["accuracy"], label="train_accuracy")
  pyplot.plot(np.arange(0, len(history.epoch)), history.history["val_accuracy"], label="val_accuracy")
  pyplot.title(""+ model + " Training and Validation Accuracy")
  pyplot.xlabel("Epoch #")
  pyplot.ylabel("Accuracy")
  pyplot.legend(loc="lower right")
  pyplot.title('Accuracy')
  pyplot.tight_layout()

  ax = pyplot.subplot(1, 2, 2)
  pyplot.plot(np.arange(0, len(history.epoch)), history.history["loss"], label="train_loss")
  pyplot.plot(np.arange(0, len(history.epoch)), history.history["val_loss"], label="val_loss")
  pyplot.title(""+ model + " Training and Validation Loss")
  pyplot.xlabel("Epoch #")
  pyplot.ylabel("Loss")
  pyplot.legend(loc="upper right")
  pyplot.tight_layout()

  pyplot.show()

# Using VGG 16

In [27]:
from keras.applications.vgg16 import VGG16, preprocess_input

In [28]:
input_shape = (224, 224, 3)
n_classes=2
conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [29]:
# Freeze base model
conv_base.trainable = False

In [30]:
from keras.models import Model
from keras.layers import *

# Create inputs with correct shape
inputs = Input(shape=(224, 224, 3))

x = conv_base(inputs, training=False)

# Add pooling layer or flatten layer
x = Flatten()(x)

# Add a hidden layer
x = Dense(256, activation='relu')(x)

# Add a dropout layer
x = Dropout(.5)(x)

# Add final dense layer
outputs = Dense(1, activation = 'softmax')(x)

# Combine inputs and outputs to create model
vgg_model_clahe = Model(inputs, outputs)
vgg_model_clahe.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (

In [31]:
vgg_model_clahe.compile(loss='binary_crossentropy',
                         metrics=["accuracy"],
                         optimizer='nadam')
checkpoint_filepath = 'vgg16_model_best_weights.h5'
# Model weights are saved at the end of every epoch, if it's the best seen so far.
vgg_checkpoint_clahe = ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)

vgg_history_clahe = vgg_model_clahe.fit(
    train_generator,
    steps_per_epoch=int(train_generator.samples / train_generator.batch_size),
    epochs=16,
    validation_data=val_generator,
    validation_steps=int(val_generator.samples / val_generator.batch_size),
    callbacks=[vgg_checkpoint_clahe]
)



Epoch 1/16


43/43 [==============================] - ETA: 0s - loss: 1.1425 - accuracy: 0.5000
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to vgg16_model_best_weights.h5
43/43 [==============================] - 471s 11s/step - loss: 1.1425 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/16
43/43 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000
Epoch 2: val_accuracy did not improve from 0.50000
43/43 [==============================] - 465s 11s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/16
15/43 [=========>....................] - ETA: 4:09 - loss: 0.6931 - accuracy: 0.5000

KeyboardInterrupt: 

In [ ]:
data_viz("VGG", vgg_history_clahe)
vgg_loss, vgg_accuracy = vgg_model_clahe.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)

In [ ]:
#Confusion Matrix and Classification Report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = vgg_model_clahe.predict(test_generator, steps=test_generator.samples/test_generator.batch_size)
y_pred = np.argmax(y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = ['freshg', 'rotteng']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
vgg_model_clahe.save('/kaggle/working/'+checkpoint_filepath)

# Using ResNet-50

In [ ]:
from tensorflow.keras.applications import ResNet101V2

In [ ]:
input_shape = (224, 224, 3)
n_classes=2
resnet_base = ResNet101V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [ ]:
# Freeze base model
resnet_base.trainable = False

In [ ]:
from keras.models import Model
from keras.layers import *

# Create inputs with correct shape
inputs = Input(shape=(224, 224, 3))

x = resnet_base(inputs, training=False)

# Add pooling layer or flatten layer
x = Flatten()(x)

# Add a hidden layer
x = Dense(256, activation='relu')(x)

# Add a dropout layer
x = Dropout(.5)(x)

# Add final dense layer
outputs = Dense(1, activation='sigmoid')(x)


# Combine inputs and outputs to create model
resnet_model = Model(inputs, outputs)
resnet_model.summary()

In [ ]:
resnet_model.compile(loss='binary_crossentropy',
                     metrics=["accuracy"],
                     optimizer='nadam')

resnet_checkpoint_filepath = 'resnet_model_best_weights.h5'
# Model weights are saved at the end of every epoch, if it's the best seen so far.
resnet_checkpoint = ModelCheckpoint(filepath=resnet_checkpoint_filepath,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)

resnet_history = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples / train_generator.batch_size,
    epochs=16,
    validation_steps=val_generator.samples / val_generator.batch_size,
    callbacks=[resnet_checkpoint]
)
)

data_viz("ResNet50", resnet_history)
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
resnet_model.save('/kaggle/working/'+resnet_checkpoint_filepath)

In [ ]:
#Confusion Matrix and Classification Report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = resnet_model.predict(test_generator, steps=test_generator.samples/test_generator.batch_size)
y_pred = np.argmax(y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = ['freshg', 'rotteng']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model

input = Input(shape=(224,224,3)) 

x  = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")(input)
x  = MaxPooling2D((2, 2))(x)

x  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(x)
x  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(x)
x  = MaxPooling2D((2, 2))(x)

x  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(x)
x  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(x)
x  = MaxPooling2D((2, 2))(x)

x  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)

x  = MaxPooling2D((2, 2))(x)


# Add pooling layer or flatten layer
x = Flatten()(x)

# Add a hidden layer
x = Dense(256, activation='relu')(x)

# Add a dropout layer
x = Dropout(.5)(x)

# Add final dense layer
outputs = Dense(2, activation='softmax')(x)
cnn_model = Model(input, outputs)
cnn_model.summary()

In [ ]:
!pip install pydot

In [ ]:
from tensorflow.keras.utils import plot_model
dot_img_file = 'C:\\Users\\siwon\\Desktop\\algo\\model_1.png'
plot_model(cnn_model, to_file=dot_img_file, show_shapes=True)

In [ ]:
cnn_model.compile(loss='categorical_crossentropy',
                  metrics=["accuracy"],
                  optimizer='nadam')

cnn_checkpoint_filepath = 'cnn_model_best_weights.h5'
# Model weights are saved at the end of every epoch, if it's the best seen so far.
cnn_checkpoint = ModelCheckpoint(filepath=cnn_checkpoint_filepath,
                                 save_weights_only=True,
                                 monitor='val_accuracy',
                                 mode='max',
                                 save_best_only=True,
                                 verbose=1)

# Convert steps_per_epoch and validation_steps to integers
steps_per_epoch = int(train_generator.samples / train_generator.batch_size)
validation_steps = int(val_generator.samples / val_generator.batch_size)

cnn_history = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=int(train_generator.samples / train_generator.batch_size),
    epochs=16,
    validation_steps=int(val_generator.samples / val_generator.batch_size),
    callbacks=[cnn_checkpoint]
)

data_viz("CNN", cnn_history)
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
cnn_model.save('/kaggle/working/'+cnn_checkpoint_filepath)


In [ ]:
#Confusion Matrix and Classification Report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = cnn_model.predict(test_generator, steps=test_generator.samples/test_generator.batch_size)
y_pred = np.argmax(y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = ['freshg', 'rotteng']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

## Performance Comparison

In [ ]:
# Graph showing accuracies and losses of the three models
pyplot.style.use("ggplot")
# sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
pyplot.plot(np.arange(0, len(vgg_history_clahe.epoch)), vgg_history_clahe.history["val_accuracy"], label="vgg16_accuracy")
pyplot.plot(np.arange(0, len(resnet_history.epoch)), resnet_history.history["val_accuracy"], label="resnet_accuracy")
pyplot.plot(np.arange(0, len(cnn_history.epoch)), cnn_history.history["val_accuracy"], label="cnn_accuracy")
pyplot.title("Models Test Accuracies")
pyplot.xlabel("Epoch #")
pyplot.ylabel("Accuracy")
pyplot.legend(loc="lower right")
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
pyplot.plot(np.arange(0, len(vgg_history_clahe.epoch)), vgg_history_clahe.history["val_loss"], label="vgg16_loss")
pyplot.plot(np.arange(0, len(resnet_history.epoch)), resnet_history.history["val_loss"], label="resnet_loss")
pyplot.plot(np.arange(0, len(cnn_history.epoch)), cnn_history.history["val_loss"], label="cnn_loss")
pyplot.title("Models Test Losses")
pyplot.xlabel("Epoch #")
pyplot.ylabel("Loss")
pyplot.legend(loc="upper right")
pyplot.tight_layout()

pyplot.show()